## Metrics on MNIST

In [1]:
from matplotlib import pyplot as plt
import numpy as np

import json
import os, sys
from os.path import join, exists

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from config import DATA_PATH

plt.rcdefaults() 
%matplotlib inline

### White box on adversarial examples frm FGSM $(\epsilon = 0.1)$, MNIST

Defences

In [2]:

CLASSIFIERS = ["cnn/relu/featsqueeze1", #Feature Squeezing
               "cnn/relu/labsmooth", # Label Smoothing
               "cnn/relu/adv-trained/fgsm/eps0.30_train.npy", # Adversarial training FGSM eps=0.4
               "cnn/relu/adv-trained/vat/eps2.10_train.npy",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.40/pert-insts10"] # Gaussian sigma =0.3 + brelu

In [3]:
INPUT_PATH = join(DATA_PATH, 'classifiers','mnist')
for c in CLASSIFIERS:
    assert exists(join(INPUT_PATH, c, "best-weights.h5"))
    assert exists(join(INPUT_PATH, c, "model.json"))
    assert exists(join(INPUT_PATH, c, "params.json"))

Adversarial Images

In [4]:
WBox_ATTACKS = join(DATA_PATH,"adversarial",'mnist')

ADV_IMAGES = ["cnn/relu/featsqueeze1", #Feature Squeezing
               "cnn/relu/labsmooth", # Label Smoothing
               "cnn/relu/adv-trained/", # Adversarial training FGSM eps=0.4
               "cnn/relu/adv-trained/vat/eps2.10_train",  # Adversarial Training VAT eps=2.1
               "cnn/brelu/gaussian/stdev0.30/pert-insts10"] # Gaussian sigma =0.3 + brelu

for c in ADV_IMAGES[0:]:
    print(c)
    assert exists(join(WBox_ATTACKS, c, "fgsm", "eps0.10_test.npy"))

cnn/relu/featsqueeze1
cnn/relu/labsmooth
cnn/relu/adv-trained/
cnn/relu/adv-trained/vat/eps2.10_train
cnn/brelu/gaussian/stdev0.40/pert-insts10


In [5]:
# Load Classifiers
from src.classifiers.cnn import CNN
from src.classifiers.utils import load_classifier
from src.utils import load_mnist
import keras.backend as K
import tensorflow as tf

#important for ensuring the model_weights are loaded in the same session as the one where metrics are computed
session = tf.Session()
K.set_session(session)

Using TensorFlow backend.


### Accuracies

In [6]:
(X_train,_), (X_test, Y_test) = load_mnist() 

In [6]:
(X_train,_), (X_test, Y_test) = load_mnist() 

# get classifier
for c,a in zip(CLASSIFIERS[0:],ADV_IMAGES[0:]):
    classifier = load_classifier(join(INPUT_PATH, c),'best-weights.h5')
    attack_imgs = np.load(join(WBox_ATTACKS,a,"fgsm", "eps0.10_test.npy"))
    print('\n',c,classifier.evaluate(x_val=attack_imgs,y_val=Y_test,batch_size=100,verbose=0)[1],'\n')
    break
    

1

 cnn/relu/featsqueeze1 0.95090000391 



## Robustness

In [9]:
from src.metrics import empirical_robustness, nearest_nieghbour_dist

In [12]:
params = {"eps_step": 0.05,
        "eps_max": 0.3,
        "clip_min": None,
        "clip_max": None}
for c,a in zip(CLASSIFIERS[0:],ADV_IMAGES):
    classifier = load_classifier(join(INPUT_PATH, c))
    emp_robust = empirical_robustness(X_test, classifier.model, session, "fgsm", params)
    print(emp_robust)
    break



1
0.142908


### Robustness as measured by proximty of decision boundary to training examples

In [7]:
from src.metrics import nearest_nieghbour_dist

params = {"eps_step": 0.05,
        "eps_max": 1.0,
        "clip_min": 0.0,
        "clip_max": 1.0}

for c,a in zip(CLASSIFIERS[0:],ADV_IMAGES):
    classifier = load_classifier(join(INPUT_PATH, c))
    avg_nn_dist = nearest_nieghbour_dist(X_test, Y_test, classifier.model, X_train,  session, "fgsm", params)
    print(c,avg_nn_dist)




1
cnn/relu/featsqueeze1 10.3774
cnn/relu/labsmooth 10.3338
cnn/relu/adv-trained/fgsm/eps0.30_train.npy 3.93939
cnn/relu/adv-trained/vat/eps2.10_train.npy 5.41566
cnn/brelu/gaussian/stdev0.40/pert-insts10 17.1036
